<a href="https://colab.research.google.com/github/TarekHasan011/Simulation-and-Modeling-Laboratory/blob/main/CSI_424(A)_Simulation_%26_Modeling_Laboratory_Assignment_4_011_181_076_Md_Tarek_Hasan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inventory System Simulation

In [ ]:
import numpy as np
import csv

In [ ]:
class Daily_Status():
    def __init__(self,cycle,days,beginning_inventory,random_digit_for_demand,demand,ending_inventory,shortage_quantity,order_quantity,random_digit_for_lead_time,days_until_order_arrive):
        self.cycle = cycle
        self.days = days
        self.beginning_inventory = beginning_inventory
        self.random_digit_for_demand = random_digit_for_demand
        self.demand = demand
        self.ending_inventory = ending_inventory
        self.shortage_quantity = shortage_quantity
        self.order_quantity = order_quantity
        self.random_digit_for_lead_time = random_digit_for_lead_time
        self.days_until_order_arrive = days_until_order_arrive

    def show(self):
        print(f'{self.cycle} {self.days} {self.beginning_inventory} {self.random_digit_for_demand} {self.demand} {self.ending_inventory} {self.shortage_quantity} {self.order_quantity} {self.random_digit_for_lead_time} {self.days_until_order_arrive}')
    def write(self,heading):
        if heading:
            f = open('inventory_system.csv','w',newline="")
            writer = csv.writer(f)
            tup1 = ("Cycle","Days", "Beginning Inventory", "Random digit for demand", "Demand", "Ending inventory", "Shortage quantity", "Order quantity","Random digit for lead time", "Days until order arrive")
            writer.writerow(tup1)
            f.close()
        f = open('inventory_system.csv','a',newline="")
        writer = csv.writer(f)
        tup1 = (self.cycle, self.days, self.beginning_inventory, self.random_digit_for_demand, self.demand, self.ending_inventory, self.shortage_quantity, self.order_quantity, self.random_digit_for_lead_time, self.days_until_order_arrive)
        writer.writerow(tup1)
        f.close()

class Inventory_System():
    def __init__(self,N,M,simulation_days,initial_inventory,current_order='-',lead_time='-'):
        self.random_number_for_demand = [24, 35, 65, 81, 54, 3, 87, 27, 73, 70, 47, 45, 48, 17, 9, 42, 87, 26, 36, 40, 7, 63, 19, 88, 94]
        self.random_number_for_lead_time = [5, 0, 3, 4, 8]
        self.N = N
        self.M = M
        self.simulation_days = simulation_days
        self.initial_inventory = initial_inventory
        self.current_order = current_order
        self.lead_time = lead_time
    
    def random_digit_assignment_for_daily_demand(self,n):
        if n>=1 and n<=10:
            return 0
        elif n>=11 and n<=35:
            return 1
        elif n>=36 and n<=70:
            return 2
        elif n>=71 and n<=91:
            return 3
        else:
            return 4
    
    def random_digit_assignment_for_lead_time(self,n):
        if n==0:
            return 3
        elif n>=1 and n<=6:
            return 1
        else:
            return 2
    
    # test random numbers
    def get_random_test_for_demand(self):
        return self.random_number_for_demand.pop(0)
    def get_random_test_for_lead_time(self):
        return self.random_number_for_lead_time.pop(0)
        

    def run(self):
        observations = []
        shortage = 0
        running_order = 0 if self.current_order == '-' else self.current_order

        for day in range(1,self.simulation_days+1):
            # random_digit_for_demand = np.random.randint(1,101)
            random_digit_for_demand = self.get_random_test_for_demand()

            random_digit_for_lead_time = '-'
            if (day%self.N) == 0:
                # random_digit_for_lead_time = np.random.randint(0,10)
                random_digit_for_lead_time = self.get_random_test_for_lead_time()

            cycle = ((day//self.N) + 1) if (day%self.N!=0) else (day//self.N)
            days = (day%self.N)
            if days == 0:
                days = self.N

            beginning_inventory = (self.initial_inventory + (self.current_order if self.lead_time==-1 else 0)) if day==1 else (observations[day-2].ending_inventory + (running_order if observations[day-2].days_until_order_arrive==0 else 0))

            b_beginning_inventory = beginning_inventory

            if shortage<=beginning_inventory:
                beginning_inventory-=shortage
                shortage = 0
            else:
                shortage-=beginning_inventory
                beginning_inventory = 0

            demand = self.random_digit_assignment_for_daily_demand(random_digit_for_demand)
            ending_inventory = max(0,beginning_inventory-demand)
            shortage_quantity = 0+shortage if beginning_inventory>=demand else (demand-beginning_inventory+shortage)
            shortage = shortage_quantity

            order_quantity = '-' 
            if days==self.N:
                order_quantity = (self.M-ending_inventory)
                running_order = order_quantity
                

            if day==1:
                if self.lead_time!='-':
                    lead_time = self.lead_time
                else:
                    lead_time = '-'
            elif days==self.N:
                lead_time = self.random_digit_assignment_for_lead_time(random_digit_for_lead_time)
            else:
                if observations[day-2].days_until_order_arrive!='-':
                    lead_time = observations[day-2].days_until_order_arrive-1
                    if lead_time<0:
                        lead_time = '-'
                else:
                    lead_time = '-'
            observations.append(Daily_Status(cycle if days==1 else '',days,b_beginning_inventory,random_digit_for_demand,demand,ending_inventory,shortage_quantity,order_quantity,random_digit_for_lead_time,lead_time))


        heading = True

        average_ending_inventory = 0
        average_shortage_days = 0

        for daily_status in observations: 
            daily_status.write(heading)
            heading = False
            average_ending_inventory += daily_status.ending_inventory
            average_shortage_days += min(daily_status.shortage_quantity,1)

        print(f'The average ending units in inventor: {average_ending_inventory/self.simulation_days}\nThe average number of days when a shortage condition occurs: {average_shortage_days/self.simulation_days}')
        return        
        

In [ ]:
obj = Inventory_System(N=5,M=11,simulation_days=25,initial_inventory=3,current_order=8,lead_time=1)
obj.run()

The average ending units in inventor: 3.52
The average number of days when a shortage condition occurs: 0.08
